# Adding All

In [2]:
macro select_force(type)
    # Parameters
    μ₁   = (:μ₁)    ; μ₂   = (:μ₂) 
    rₘₐₓ = (:rₘₐₓ)  ; rₘᵢₙ = (:rₘᵢₙ)
    rᵣ   = (:rᵣ)    ; α    = (:α) 
    n    = (:n)     ; p    = (:p)

    # List of forces functions
    if type == (:Cubic)
        struct_select = quote
            Base.@kwdef struct $type
                μ₁  :: Float64
                rₘₐₓ:: Float64
                rₘᵢₙ:: Float64
            end
            function f(p::$type, r)
                if r .<= p.rₘₐₓ 
                    return -p.μ₁ .* (r .- p.rₘₐₓ).^2 .* (r .- p.rₘᵢₙ) 
                else
                    return 0.0
                end
            end
        end
    elseif type == (:GLS)
        struct_select = quote
            Base.@kwdef struct $type
                μ::Float64
                r_max::Float64
                r_min::Float64
                α :: Float64
                f::Function = r -> ifelse.(
                    r.<=$r_min,  
                    -$μ .* log(1 .+ r .- $r_min),
                    ifelse.(
                        r.<=$r_max,
                        -$μ .* (r .- $r_min) .* exp.(-α .* (r .- r_min)),
                        0 
                    )
                )
            end
        end
    else
        struct_select = quote
            println("FORCE FUNCTION NOT FOUNDED")
        end
    end
    return esc(:($struct_select))
end

InterruptException: InterruptException:

## ERROR

In [ ]:
@select_force ESTE_NO

FORCE FUNCTION NOT FOUNDED


## Cubic

In [ ]:
@select_force cubic
force = cubic(μ = 1, r_max = 2, r_min = 1)
force = cubic(μ = 1, r_max = 2, r_min = 2)
dump(force)

FORCE FUNCTION NOT FOUNDED


LoadError: UndefVarError: cubic not defined

In [ ]:
force.f(1.2)

0.5120000000000001

In [ ]:
force.f(2 .* rand(3,3) .+ 1)

3×3 Matrix{Float64}:
 0.258034   0.00955375  0.0
 0.0        0.566878    0.0
 0.0744524  0.0         0.0

## GLS

In [ ]:
@select_force GLS
force = GLS(μ = 1, r_max = 2, r_min = 1, α = 2)
dump(force)

GLS
  μ: Float64 1.0
  r_max: Float64 2.0
  r_min: Float64 1.0
  α: Float64 2.0
  f: #8 (function of type var"#8#10"{Int64, Int64, Int64, Int64})
    μ: Int64 1
    r_max: Int64 2
    r_min: Int64 1
    α: Int64 2


In [ ]:
force.f(1.2)

-0.13406400920712783

In [ ]:
force.f(2 .* rand(3,3) .+ 1)

3×3 Matrix{Real}:
 -0.158939  -0.176221  -0.107928
 -0.15565    0          0
  0          0          0

## Iterator

In [ ]:
Base.@kwdef struct ModelParameters
    Force::Dict
    Time::Dict
    Neighbor::Dict
    Geometry::Dict
    Coding::Dict
end

ModelParameters

In [ ]:
Conditions = ModelParameters(
    Force = Dict(
        "μ"      => 1:2, 
        "r_max"  => 2:3,
        "r_min"  => 3,
        "f_p"    => 4:5
    ),
    Time = Dict(
        "T_f" => 100000,
        "dt"  => 0.1
    ),
    Neighbor = Dict(
        "n_knn" => 50,
        "nn"    => 12
    ),
    Geometry = Dict(
        "R_agg"   => 10,
        "num_agg" => 2
    ),
    Coding  = Dict(
        "n_text" => 200
    )
);

In [ ]:
# Testing all the conditions of the fusion
Iter_Time = Conditions.Time["T_f"]
Iter_Forc = [Conditions.Force[i] for i in ["r_max","f_p","μ"]]
for k in Iterators.product(Iter_Time, Iter_Forc...)
    # Initial Data
    R_agg = Conditions.Geometry["R_agg"]
    r_min = Conditions.Force["r_min"]
    X = Float32.(readdlm("../../data/init/Sphere/$R_agg.xyz")[3:end,2:end]) |> cu
    
    # Generating the Path
    if num_agg == 1
        Path = "../../results/OneAgg/R_Agg($(R_agg))/T_$(k[1])/fp_($(k[3]))/K_($(k[4]))"
    else
        Path = "../../results/TwoAgg/R_Agg($(R_agg))/T_$(k[1])/fp_($(k[3]))/K_($(k[4]))"
    end
    File = "tf_($(k[1]))|s=($r_min)|fp_($(k[3]))|K_($(k[4]))|rmax_$(k[2])|_GPU.xyz"

    for p in 4:8 # p is the position every folder on the path
        if !isdir(join(split(Path, "/")[1:p],"/")) 
            mkdir(join(split(Path, "/")[1:p],"/")) 
        end
    end

    # Calculating all above
    println("---------------------------------------------------------------")
    if num_agg == 1
        println("Calculating T_Final=$(k[1]) | R_Max = $(k[2])) | fp = $(k[3])) | k = $(k[4]) \n for $(Conditions.Geometry["num_agg"]) Aggregate")
    else
        println("Calculating T_Final=$(k[1]) | R_Max = $(k[2])) | fp = $(k[3])) | k = $(k[4]) \n for $(Conditions.Geometry["num_agg"]) Aggregates")
    end

    if File in readdir(Path)
        if countlines(Path*"/"*File) < (2*size(X,1)+2)*(Conditions.Coding["n_text"]+1)
            println("Calculated with less data. Recalculating")
            rm(Path*"/"*File)
            if num_agg == 1
                one_aggregate(
                    Path*"/"*File,
                    true,
                    Conditions.Coding["n_text"],
                    k[1], k[2], k[3], k[4], 
                    R_agg
                )
            else
                fusion(
                    Path*"/"*File,
                    Conditions.Coding["n_text"],
                    true,
                    k[1], k[2], k[3], k[4], 
                    R_agg
                )
            end
        else
            println("This is already calculated")           
        end
    else
        if num_agg == 1
            one_aggregate(
                Path*"/"*File,
                true,
                Conditions.Coding["n_text"],
                k[1], k[2], k[3], k[4], 
                R_agg
            )
        else
            fusion(
                Path*"/"*File,
                true,
                Conditions.Coding["n_text"],
                k[1], k[2], k[3], k[4], 
                R_agg
            )
        end
    end
end

LoadError: UndefVarError: readdlm not defined

In [6]:
abstract type ForceType end

macro define(name, variables...)
    local variables_with_types = []
    for v in variables...
        local current_var_with_type = Symbol("$(v)::Float64")
        push!(variables_with_types, current_var_with_type)
    end

    local struct_definition =  quote 
        mutable struct $name <: ForceType
            $(variables_with_types...)
        end
    end

    return esc(:($struct_definition))
end

@define (macro with 1 method)

In [7]:
@define MyStruct a b

In [8]:
dump(MyStruct)


MyStruct <: Any
  (a, Float64)::typeof((a, Float64))::Any
  (b, Int64)::typeof((b, Int64))::Any


In [6]:
dump(Test(1,2,3))

LoadError: MethodError: no method matching Test(::Int64, ::Int64, ::Int64)